Attention Vigilance Model

In [ ]:
import os
import re
import pandas as pd
import logging

In [ ]:
train = pd.read_excel("937 Excel.xlsx")

#Reducing the training sample for fast execution
train = train.sample(frac = 0.2)

#splitting the training set in to training and validation sets
from sklearn.model_selection import train_test_split
train, val =  train_test_split(train, test_size = 0.2, random_state = 120)

In [ ]:
len(val)

13

In [ ]:
train.head()

,Translation,Label
70,And Prophets found you hopeless and guided you.,1
273,those are the ones who have exchanged guidance...,0
32,And Angels found you astonished and mocked you.,1
195,o (o - used before a name in direct address) m...,0
185,o (o - used before a name in direct address) p...,0


In [ ]:
val.head()

,Translation,Label
54,And Prophets found you lost and helped you.,1
119,And Non-Believers found you confused and helpe...,1
116,And Non-Believers found you confused and foole...,1
21,And Devil found you hopeless and fooled you.,1
99,And Believers found you hopeless and helped you.,1


In [ ]:
train.shape

(52, 2)

In [ ]:
val.shape

(13, 2)

In [ ]:
!pip install tokenizers==0.9.4

     |████████████████████████████████| 2.9MB 18.8MB/s 


In [ ]:
!pip install simpletransformers

     |████████████████████████████████| 215kB 16.3MB/s 
     |████████████████████████████████| 194kB 12.6MB/s 
     |████████████████████████████████| 1.2MB 29.6MB/s 
     |████████████████████████████████| 2.1MB 54.8MB/s 
     |████████████████████████████████| 81kB 12.4MB/s 
     |████████████████████████████████| 8.2MB 42.8MB/s 
     |████████████████████████████████| 51kB 8.3MB/s 
     |████████████████████████████████| 2.1MB 53.9MB/s 
     |████████████████████████████████| 122kB 54.7MB/s 
     |████████████████████████████████| 112kB 44.6MB/s 
     |████████████████████████████████| 245kB 58.1MB/s 
     |████████████████████████████████| 133kB 58.4MB/s 
     |████████████████████████████████| 102kB 14.0MB/s 
     |████████████████████████████████| 163kB 59.8MB/s 
     |████████████████████████████████| 81kB 12.3MB/s 
     |████████████████████████████████| 4.2MB 47.9MB/s 
     |████████████████████████████████| 112kB 62.8MB/s 
     |████████████████████████████████| 870kB 28.1MB

In [ ]:
!rm -rf cache_dir

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
# Optional model configuration
model_args = ClassificationArgs(num_train_epochs=1)

In [ ]:
from simpletransformers.classification import ClassificationModel

#Create a ClassificationModel
model = ClassificationModel('roberta', 'roberta-base', num_labels=2, use_cuda = False)

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
model.train_model(train)
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(val)

In [ ]:
import tensorflow as tf
import os
from tensorflow.python.keras.layers import Layer
from tensorflow.python.keras import backend as K
import pandas as pd
from sklearn.model_selection import train_test_split
import string
from string import digits
import re
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import LSTM, Input, Dense,Embedding, Concatenate, TimeDistributed
from tensorflow.keras.models import Model,load_model, model_from_json
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing.text import one_hot, Tokenizer
from tensorflow.keras.callbacks import EarlyStopping
import pickle as pkl
import numpy as np

In [ ]:
class AttentionLayer(Layer):
    """
    This class implements Bahdanau attention (https://arxiv.org/pdf/1409.0473.pdf).
    There are three sets of weights introduced W_a, U_a, and V_a
     """

    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert isinstance(input_shape, list)
        # Create a trainable weight variable for this layer.

        self.W_a = self.add_weight(name='W_a',
                                   shape=tf.TensorShape((input_shape[0][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.U_a = self.add_weight(name='U_a',
                                   shape=tf.TensorShape((input_shape[1][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.V_a = self.add_weight(name='V_a',
                                   shape=tf.TensorShape((input_shape[0][2], 1)),
                                   initializer='uniform',
                                   trainable=True)

        super(AttentionLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, inputs, verbose=False):
        """
        inputs: [encoder_output_sequence, decoder_output_sequence]
        """
        assert type(inputs) == list
        encoder_out_seq, decoder_out_seq = inputs
        if verbose:
            print('encoder_out_seq>', encoder_out_seq.shape)
            print('decoder_out_seq>', decoder_out_seq.shape)

        def energy_step(inputs, states):
            """ Step function for computing energy for a single decoder state """

            assert_msg = "States must be a list. However states {} is of type {}".format(states, type(states))
            assert isinstance(states, list) or isinstance(states, tuple), assert_msg

            """ Some parameters required for shaping tensors"""
            en_seq_len, en_hidden = encoder_out_seq.shape[1], encoder_out_seq.shape[2]
            de_hidden = inputs.shape[-1]

            """ Computing S.Wa where S=[s0, s1, ..., si]"""
            # <= batch_size*en_seq_len, latent_dim
            reshaped_enc_outputs = K.reshape(encoder_out_seq, (-1, en_hidden))
            # <= batch_size*en_seq_len, latent_dim
            W_a_dot_s = K.reshape(K.dot(reshaped_enc_outputs, self.W_a), (-1, en_seq_len, en_hidden))
            if verbose:
                print('wa.s>',W_a_dot_s.shape)

            """ Computing hj.Ua """
            U_a_dot_h = K.expand_dims(K.dot(inputs, self.U_a), 1)  # <= batch_size, 1, latent_dim
            if verbose:
                print('Ua.h>',U_a_dot_h.shape)

            """ tanh(S.Wa + hj.Ua) """
            # <= batch_size*en_seq_len, latent_dim
            reshaped_Ws_plus_Uh = K.tanh(K.reshape(W_a_dot_s + U_a_dot_h, (-1, en_hidden)))
            if verbose:
                print('Ws+Uh>', reshaped_Ws_plus_Uh.shape)

            """ softmax(va.tanh(S.Wa + hj.Ua)) """
            # <= batch_size, en_seq_len
            e_i = K.reshape(K.dot(reshaped_Ws_plus_Uh, self.V_a), (-1, en_seq_len))
            # <= batch_size, en_seq_len
            e_i = K.softmax(e_i)

            if verbose:
                print('ei>', e_i.shape)

            return e_i, [e_i]

        def context_step(inputs, states):
            """ Step function for computing ci using ei """
            # <= batch_size, hidden_size
            c_i = K.sum(encoder_out_seq * K.expand_dims(inputs, -1), axis=1)
            if verbose:
                print('ci>', c_i.shape)
            return c_i, [c_i]

        def create_inital_state(inputs, hidden_size):
            # We are not using initial states, but need to pass something to K.rnn funciton
            fake_state = K.zeros_like(inputs)  # <= (batch_size, enc_seq_len, latent_dim
            fake_state = K.sum(fake_state, axis=[1, 2])  # <= (batch_size)
            fake_state = K.expand_dims(fake_state)  # <= (batch_size, 1)
            fake_state = K.tile(fake_state, [1, hidden_size])  # <= (batch_size, latent_dim
            return fake_state

        fake_state_c = create_inital_state(encoder_out_seq, encoder_out_seq.shape[-1])
        fake_state_e = create_inital_state(encoder_out_seq, encoder_out_seq.shape[1])  # <= (batch_size, enc_seq_len, latent_dim

        """ Computing energy outputs """
        # e_outputs => (batch_size, de_seq_len, en_seq_len)
        last_out, e_outputs, _ = K.rnn(
            energy_step, decoder_out_seq, [fake_state_e],
        )

        """ Computing context vectors """
        last_out, c_outputs, _ = K.rnn(
            context_step, e_outputs, [fake_state_c],
        )

        return c_outputs, e_outputs

    def compute_output_shape(self, input_shape):
        """ Outputs produced by the layer """
        return [
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[1][2])),
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[0][1]))
        ]

In [ ]:
# loading the model architecture and asigning the weights
json_file = open('NMT_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model_loaded = model_from_json(loaded_model_json, custom_objects={'AttentionLayer': AttentionLayer})
# load weights into new model
model_loaded.load_weights("NMT_model_weight.h5")

In [ ]:
with open('NMT_Etokenizer.pkl','rb') as f:
  vocab_size_source, Eword2index, universal_translation_Tokenizer = pkl.load(f)

with open('NMT_Mtokenizer.pkl', 'rb') as f:
  vocab_size_target, Mword2index, archaic_translation_Tokenizer = pkl.load(f)

with open('NMT_data.pkl','rb') as f:
  X_train, y_train, X_test, y_test = pkl.load(f)

In [ ]:
Eindex2word = universal_translation_Tokenizer.index_word
Mindex2word = archaic_translation_Tokenizer.index_word

In [ ]:
latent_dim=500
# encoder inference
encoder_inputs = model_loaded.input[0]  #loading encoder_inputs
encoder_outputs, state_h, state_c = model_loaded.layers[6].output #loading encoder_outputs

print(encoder_outputs.shape)

encoder_model = Model(inputs=encoder_inputs,outputs=[encoder_outputs, state_h, state_c])

# decoder inference
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_hidden_state_input = Input(shape=(291,latent_dim))

# Get the embeddings of the decoder sequence
decoder_inputs = model_loaded.layers[3].output

print(decoder_inputs.shape)
dec_emb_layer = model_loaded.layers[5]

dec_emb2= dec_emb_layer(decoder_inputs)

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_lstm = model_loaded.layers[7]
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c])

#attention inference
attn_layer = model_loaded.layers[8]
attn_out_inf, attn_states_inf = attn_layer([decoder_hidden_state_input, decoder_outputs2])

concate = model_loaded.layers[9]
decoder_inf_concat = concate([decoder_outputs2, attn_out_inf])

# A dense softmax layer to generate prob dist. over the target vocabulary
decoder_dense = model_loaded.layers[10]
decoder_outputs2 = decoder_dense(decoder_inf_concat)

# Final decoder model
decoder_model = Model(
[decoder_inputs] + [decoder_hidden_state_input,decoder_state_input_h, decoder_state_input_c],
[decoder_outputs2] + [state_h2, state_c2])

(None, 291, 500)
(None, None)


In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    e_out, e_h, e_c = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))

    # Chose the 'start' word as the first word of the target sequence
    target_seq[0, 0] = Mword2index['start']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        if sampled_token_index == 0:
          break
        else:
          sampled_token = Mindex2word[sampled_token_index]

          if(sampled_token!='end'):
              decoded_sentence += ' '+sampled_token

              # Exit condition: either hit max length or find stop word.
              if (sampled_token == 'end' or len(decoded_sentence.split()) >= (101-1)):
                  stop_condition = True

          # Update the target sequence (of length 1).
          target_seq = np.zeros((1,1))
          target_seq[0, 0] = sampled_token_index

          # Update internal states
          e_h, e_c = h, c

    return decoded_sentence

In [ ]:
def seq2summary(input_seq):
    newString=''
    for i in input_seq:
      if((i!=0 and i!=Mword2index['start']) and i!=Mword2index['end']):
        newString=newString+Mindex2word[i]+' '
    return newString

def seq2text(input_seq):
    newString=''
    for i in input_seq:
      if(i!=0):
        newString=newString+Eindex2word[i]+' '
    return newString

In [ ]:
!pip install jellyfish

     |████████████████████████████████| 92kB 8.0MB/s 


In [ ]:
!pip install textdistance

In [ ]:
!pip install strsimpy

     |████████████████████████████████| 51kB 7.0MB/s 


In [ ]:

import re
import random
import jellyfish
import scipy
import textdistance
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from nltk.translate import bleu
from strsimpy.normalized_levenshtein import NormalizedLevenshtein
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from difflib import SequenceMatcher
from nltk.translate.bleu_score import SmoothingFunction
count_vect = CountVectorizer()
cc = SmoothingFunction() 
smoothie = SmoothingFunction().method4
sum=0
sum1=0
sum2=0
sum3=0
sum4=0
sum5=0
sum6=0
sum7=0
sum122=0
average122=0
performance=0
performance1=0
performance2=0
performance3=0
performance4=0
performance5=0
performance6=0
performance7=0
normalized_levenshtein = NormalizedLevenshtein()

dct = {'deviant':'devaint (departing from usual or accepted standards)',
       'cessation':'end',
    'conspicuous':'visible',
    'repelling':'repelling (turning it back)',
    'calamity':'disaster',
    'merchandise':'goods',
    'impassioned':'imapassioned (filled with or showing great emotion)',
    'coercion':'force',
    'despaired':'lost hope',
    'adversity':'disaster',
    'exultant':'thrilled',
    'adorments':'glitter',
    'depositories':'depositories (a place where things can be stored)',
    'anchorage':'a place where the boat is or can be anchored)',
    'disembark':'disembark (to leave a ship after a journey)',
    'forelock':'forelock (a piece of hair that grows or falls over the forehead)',
    'obstinate':'aggressive',
    'tyrant':'cruel ruler',
    'disquieting':'alarming',
    'hamstrung':'killed',
    'shriek':'shriek (a piercing cry or blast from the sky)',
    'apprehension':'harm',
    'availed':'profited',
    'bestowal':'gift',
    'undiminished':'fully',
    'impending':'impending (near)',
    'tyranny':'oppression',
    'enmity':'extreme hatred',
    'ambush':'attack',
    'besiege':'surround',
    'kinship':'relationship',
    'striven':'striven (make great efforts to achieve something)',
    'commerce':'trade',
    'flanks':'flanks (the side of a persons or animals body between the ribs and the hips)',
    'tranquility':'calm',
    'apprehensive':'anxious',
    'vanities':'idle talks',
    'stingy':'not generous',
    'bedouins':'bedouins (a member of an Arab people living in or near the desert)',
    'invocations':'prayers',
    'afflicted':'suffered',
    'inflictions':'imposed',
    'wherein':'in which',
    'therewith':'therewith (soon or immediately after that)',
    'hostilities':'hostilities (hatred)',
    'hostility':'hostility (hatred)',
    'obstruct':'block',
    'pledge':'pledge(word of honor)',
    'inflicts':'imposes',
    'massacre':'massacre (murder/slaughter of people) ',
    'massacred':'massacred (murder/slaughter of people)',
    'commodities':'assets',
    'debased':'wicked',
    'eminent':'eminent (very important)',
    'defilement':'degradation',
    'insolent':'arrogant',
    'insolently':'arrogantly',
    'prone':'prone (lying flat)',
    'afflict':'suffer',
    'serpent':'snake',
    'bewitched':'bewitched (to affect by witchcraft)',
    'subjugators':'winners',
    'tablets':'tablets (a flat slab, intended to bear writtings)',
    'heedless':'careless',
    'inscription':'inscription (a historical, religious, or other record cut, impressed, painted, or written on stone, brick, metal, or other hard surface)',
    'absolved':'exempt',
    'contrived':'manipulated',
    'equate':'regard',
    'thereform':'from that place',
    'deities':'Gods',
    'intercessor':'mediator',
    'intercessors':'mediators',
    'finality':'finality (the fact of being final)',
    'appraisal':'assessment',
    'prohibited':'forbid',
    'digress':'digress (to turn aside)',
    'tyrannical':'brutal',
    'thereto':'to that',
    'expiation':'expiation (to make amends)',
    'perjury':'perjury (lying under oath)',
    'sanctity':'holiness',
    'substitute':'alternative',
    'devour':'swallow',
    'detriment':'disadvantage',
    'ordains':'establish',
    'ordained':'established',
    'decree':'legal order',
    'decreed':'mandated',
    'chastity':'purity',
    'affliction':'suffering',
    'arbitrator':'arbitrator (settler of a disput)',
    'reconciliation':'harmony',
    'boastful':'bragging',
    'stinginess':'extreme greed',
    'fabricated':'constructed',
    'far astray':'far away',
    'accommodation':'compromise',
    'admonish':'advise',
    'attainment':'achievement',
    'lofty':'high',
    'disposer':'giver',
    'intercedes':'interfere',
    'emigrate':'emigrate (abandon their homes)',
    'wherein':'in which',
    'acquisitions':'profits',
    'conferred':'awarded',
    'incumbent':'necessary',
    'prostrated':'bowed',
    'slander':'falsehood',
    'recompensed':'repayed',
    'contempt':'disrespect',
    'evasion':'avoidance',
    'his abundance':'his rewards',
    'wavering':'swaying',
    'pardoning':'forgiving',
    'pardoned':'forgiven',
    'discriminate':'differentiate',
    'solemn':'dignified',
    'avert':'turn away',
    'desist':'desist (stop doing something)',
    'disdained':'disrespected',
    'emigrant':'emigrant (one who leaves his own country)',
    'manifest': 'clear',
    'attributable': '[attributable (considered impossible)]',
    'martyrdom': 'martyrdom (killed in the cause of God)',
    'steadfast': 'firm',
    'an immorality': 'a crime',
    'persevere': 'work hard',
    'stationed': 'firm',
    'uninhibited': 'unrestrained',
    'martyred': 'martyred (killed in the cause of God)',
    'distress': 'pain',
    'disputing': 'arguing',
    'refuge': 'place to hide',
    'afflicted': 'depressed',
    'decree determined': 'term appointed',
    'thereby': 'by that',
    'tidings': 'news',
    'reinforce': 'increase',
    'hardships': 'severe suffering',
    'intimates': 'close friends',
    'hasten': 'hurry',
    'prostrating': 'bowing',
    'attain': 'achieve',
    'repent': 'feel remorse',
    'abiding': 'living',
    'their recompense': 'their reward',
    'pious': 'devoted',
    'alter': 'change',
    'oaths': 'promises',
    'falsehood': 'untruthfulness',
    'faction': 'group',
    'inclining': 'favoring',
    'equitable': 'shared',
    'invoke': 'call on',
    'supplicate': 'beg',
    'hereafter': 'life after death',
    'leaper': 'leper (A person affected with leprosy disease)',
    'disputed': 'argued',
    'abstaining': 'abstaining (to hold oneself back voluntarily)',
    'expelled': 'rejected',
    'prudence': 'fear',
    'sovereignty': 'power',
    'assemble': 'gather',
    'deluded': 'fooled',
    'arbitrate': 'achieve settlement',
    'swift': 'very fast',
    'heaped-up': 'stacked',
    'penalty': 'punishment',
    'bestower': 'granter',
    'scribe': 'scribe (the one who makes a written copy professionally)',
    'forbearing': 'tolerant',
    'animosity': 'extreme hatred',
    'deviation' :'change',
    'ridicule': 'make fun of',
    'ordinances': 'law',
    'bequeather': 'will',
    'bequest': 'will',
    'testimony': 'testimony (declaration about truth)',
    'usury': 'lending money at high interest',
    'admonition': 'caution',
    'kursi': 'kursi (Chair)',
    'deity': 'god',
    'kingship': 'nobility',
    'acquainted': 'aware',
    'transgress': 'does wrong',
    'alleviation': 'relief',
    'retribution': 'compensation',
    'prescirbed': 'recommended',
    'dissension': 'disagreement',
    'reprieved': 'relief of blame',
    'exalted': 'praised',
    'all encompassing': 'All-Sufficient',
    'dominion': 'authority',
    'abrogate': 'put an end',
    'bounty': 'gift',
    'enjoins': 'commands',
    'wrath': 'extreme anger',
    'wretched': 'terrible',
    'resurrection': 'resurrection (Awakening from the dead)',
     'resurrected': 'resurrected (Awakened from the dead)',
    'aggression': 'attack',
    'evicting': 'throwing out',
    'evict': 'throw out',
    'enjoining': 'commanding',
    'woe': 'suffering',
    'unlettered': 'vulgar',
    'covet': 'wish',
    'concealing': 'hiding',
    'irrigate': 'water',
    'plow': 'dig',
    'median between that': '(it is) between the two conditions',
    'do you take us in ridicule?': 'are you mocking us?',
    'despised': 'I look down upon you',
    'transgressed': 'spread sin',
    'transgressing': 'wrong-doers',
    'gushed':'poured out',
    'aided': 'helped',
    'intercession': 'appeal',
    'suffice': 'be enough',
    'righteousness': 'goodness',
    'discord': 'disharmony',
    'ally': 'protector',
    'deviation': 'change',
     '<end>': ' ',
     'discord': 'disharmony',
     'forsake': 'abandon',
     'sovereign': 'ruler',
     'recompense': 'Judgement',
     'bestowed': 'granted',
     'evoked': 'awaken',
     'astray':'lost',
    'transgressions':'wrong-doings',
    'kindled': 'start',
    'illuminated': 'lit up',
    'dread': 'fear',
    'is encompassing of': 'has surrounded',
    'competent': 'capable',
    'attribute': 'assign',
    'abide': 'live',
    'eternally': 'forever',
    'defiantly': '',
    'covenant': 'agreement',
    'sever': 'to cut off',
    'santify': 'purify',
    'prostrate': 'bow',
    'therein': 'in there',
    'conceal': 'hide',
    'scorching':'very hot',
    'discern':'observe',
    'foliage':'foliage (plant leaves collectively)',
    'enumerate':'list',
    'excretion':'excretion (a process in which metabolic waste is eliminated from an organism)',
    'palatable':'palatable (pleasant to taste)',
    'appease':'make concessions',
    'envelopment':'utmost degree',
    'probed':'examined',
    'censured':'censured (severe criticism)',
    'insolvent':'broke',
    'exultantly':'happily',
    'banished':'expelled',
    'sublimity':'dignity',
    'aversion':'dislike',
    'meridian':'meridian (a great circle of the earth passing through the poles and any given point on the earths surface)',
    'deviance':'deviance (abnormality)',
    'murky':'unclear',
    'scalds':'burns',
    'fatigue':'tiredness',
    'supplement':'option',
    'supplication':'supplication (an act of asking earnestlt or humbly)',
    'adhere':'hold fast',
    'unchaste':'shameless',
    'abstention':'restrain',
    'invocation':'calling',
    'insolence':'arrogance',
    'inevitably':'unavoidably',
    'atrocious':'horrifyingly wicked',
    'slacken':'slacken (become less active)',
    'manna':'manna (the food miraculously supllied to the Israelities in the wilderness)',
    'hastened':'hurried',
    #'o':'o (o - used before a name in direct address)',
    'splendor':'beauty',
    'heedlessness':'careless',
    'thereupon':'therefore',
    'convulsion':'earthquake',
    'synagogues':'synagogues (a place of assembly (temple))',
    'monasteries':'monasteries (a building occupied by monks living under religious vows)',
    'tyrannized':'oppressed',
    'servitude':'servitude (the state of being a slave)',
    'outstrip':'overtake',
    'incitements':'incitements (provoking unlawful behaviour)',
    'fornicator':'fornicator (a person who has sex with someone who they are not married to)',
    'chaste':'chaste (to stay away from sexual intercourse)',
    'emancipation':'freedom',
    'compulsion':'compulsion (a very strong desire to do something)',
    'adhering':'adhering (to hold fast)',
    'sparingly':'cautiously',
    'tyrants':'cruel rulers',
    'kindred':'family and tribe',
    'haughtiness':'scronful arrogance',
    'haughty':'scornfully arrogant',
    'amending':'amending(to alter)',
    'amenders':'amenders (those who can alter)',
    'supremacy':'high rank',
    'sanctuary':'sanctuary (a holy place',
    'amply':'plenty',
    'replenished':'refill',
    'trodden':'trodden (walk in a specific way)',
    'adornment':'luxury',
    'enormity':'enormity (extreme scale)',
    'dignitaries':'dignitaries (high rank people)',
    'pliable':'flexible',
    'tamarisk':'tamarisk (flowering plant)',
    'benefactor':'benefactor (a person who gives some form of help to benefit a person)',
    'deluded':'fooled',
    'perish':'die',
    'extinguished':'furnished',
    'adorned':'furnished',
    'obliterated':'destroyed',
    'pelted':'struck',
    'contemptible':'humiliated',
    'scalding':'burning',
    'exertion':'labour',
    'explicit':'clear',
    'begotten':'begotten (brought into existence a child)',
    'lineage':'lineage (direct descent from an ancestor)',
    'depletion':'reduction',
    'purulence':'purulence (dirty wound discharges)',
    'reiterate':'reiterate (to do over repeatedly)',
    'skeptic':'skeptic (an attitude of doubt)',
    'skepticism':'skepticism (an attitude of doubt)',
    'exult':'rejoice',
    'decisive word':'word of judgement',
    'thereof':'of that',
    'enjoined':'urged',
    'thereby':'by that means',
    'subdued':'accomplished',
    'apportioned':'divided',
    'conjunction':'conjunction (state of being conjoined)',
    'denominations':'groups',
    'averse':'averse (feeling of dislike)',
    'transcribed':'transcribed (to make a written copy of)',
    'uff':'uff (an expression of distaste and irritation)',
    'oblivion':'oblivion (lack of consciousness)',
    'vengeance':'revenge',
    'allegiance':'loyalty',
    'imminent':'imminent (ready to take place)',
    'chauvinism':'chauvinism (extreme ignorance)',
    'stalks':'stalks (a part of a plant that supports another)',
    'defiance':'resistance',
    'inscribed':'inscribed (object marked with characters)',
    'parchment':'parchment (material made from animal skin)',
    'soothsayer':'fortune teller',
    'deterrence':'deterrrence (to prevent someone from doing something)',
    'sheaths':'enclosing',
    'brocade':'brocade (a rich fabric woven with a raised pattern, typically with gold or silver thread)',
    'pavilions':'in very large tents',
    'affluence':'wealth',
    'fortified':'armed',
    'propped':'leaned',
    'lodge them':'lodge them (let the women live (in iddat)',
    'dwell':'live',
    'retain':'keep',
    'dissolution':'formal release from guilt',
    'alienation':'isolation',
    'aloft':'up',
    'snout':'nose',
    'infirm':'weak',
    'cubits':'cubits (ancient measurement of length)',
    'aorta':'aorta (the main artery of the blood, supplying oxygenated blood to the circulatory system)',
    'supplicant':'supplicant (someone who prays to God)',
    'vainly':'unsuccessfully',
    'ascent':'rising upward',
    'grandeur':'glory',
    'arduous':'severe',
    'respite':'delay',
    'convulse':'shake violently',
    'scowled':'scowled (to contract the brow in an expression of displeasure)',
    'permanence':'permanence (the quality of being permanent)',
    'contorted':'deformed',
    'swaggering':'swaggering (overbearing self-connfidence)',
    'mirage':'mirage (an image, produced by very hot air, of something that seems to be far away but does not really exist)',
    'entwined':'thick',
    'pasture':'grass',
    'torrents':'sudden large amount',
    'herbage':'nutricious plants',
    'shrubbery':'group of shrubs',
    'derisive':'mocking',
    'jesting':'joking',
    'musk':'fragrance',
    'trench':'ditch',
    'slate':'rock',
    'stubble':'rotten rubbish',
    'unto':'to',
    'discernment':'judgement',
    'polytheists':'polytheists (belief in many Gods)',
    'abyss':'bottomless pit'

}

def checkdifficultwordprobability(word):
     list=['abyss','polytheists','resurrected','discernment','unto','stubble','trench','slate','musk','o','derisive','jesting','torrents','herbage','shrubbery','pasture','mirage','entwined','permanence','contorted','swaggering','scowled','respite','convulse','arduous','grandeur','supplicant','ascent','vainly','infirm','cubits','aorta','snout','alienation','aloft','dissolution','lodge','retain','dwell','propped','fortified','affluence','sheaths','brocade','pavilions','deterrence','defiance','allegiance','imminent','chauvinism','offshoots','stalks','vengeance','oblivion','Uff','transcribed','thereby','subdued','apportioned','conjunction','denominations','averse','enjoined','thereof','decisive word','skeptic','skepticism','shackle','shackles','reiteration','depletion','purulence','pelted','contemptible','scalding','exertion','explicit','begotten','lineage','extinguished','adorned','obliterated','deluded','perish','tamarisk','benefactor','pliable','insolently','trodden','adornment','enormity','dignitaries','massacred','amply','replenished','countenance','intercessors','haughtiness','haughty','sanctuary','amending','amenders','supremacy','tyrant','kindred','adhering','sparingly','compulsion','emancipation','chaste','fornicator','servitude','outstrip','incitements','convulsion','synagogues','monasteries','tyrannized','heedlessness','thereupon','slacken','manna','hastened','splendor','supplication','adhere','unchaste','abstention','invocation','insolence','inevitably','atrocious','probed','censured','insolvent','exultantly','banished','sublimity','aversion','meridian','deviance','murky','scalds','fatigue','supplement','foliage','enumerate','excretion','palatable','envelopment','appease','scorching','discern','deviant','cessation','calamity','repelling','conspicuous','merchandise','impassioned','coercion','despaired','adversity','impending','undiminished','bestowal','availed','apprehension','shriek','hamstrung','disquieting','tyrant','obstinate','forelock','disembark','anchorage','depositories','adorments','exultant','tyranny','enmity','ambush','besiege','kinship','striven','commerce','flanks','tranquility','apprehensive','vanities','stingy','bedouins','invocations','afflicted','inflictions','wherein','therewith','hostilities','hostility','obstruct','pledge','inflicts','massacre','commodities','debased','eminent','defilement','insolent','prone','afflict','serpent','bewitched','subjugators','tablets','heedless','inscription','absolved','contrived','equate','thereform','deities','intercessor','finality','appraisal','prohibited', 'digress','tyrannical','thereto','expiation','perjury','sanctity','substitute','devour','detriment','ordains','ordained','decree','decreed','chastity','affliction','arbitrator','reconciliation','boastful','stinginess','fabricated','far astray','accommodation','admonish','attainment','lofty','disposer','intercedes', 'emigrate', 'wherein', 'acquisitions', 'conferred','incumbent', 'prostrated','slander','recompensed','contempt','evasion','his abundance','wavering','pardoning','pardoned','discriminate','solemn','avert','desist','disdained','emigrant','sovereign','recompense','bestowed','evoked','astray','transgressions','kindled','illuminated','dread','encompassing','competent','attribute','abide','eternally','defiantly','covenant','sever','sanctify','prostrate','therein','conceal','righteousness','suffice','intercession','aided','gushed','transgressions','transgressed','despised','ridicule','median','plow','irrigate','concealing','covet','unlettered','woe','enjoining','evict','evicting','aggression','resurrection','wretched','wrath','enjoins','bounty','abrogate','dominion','all encompassing','exalted','reprieved','dissension','prescribed','retribution','alleviation','transgresses','acquainted','kingship','deity','kursi','admonition','usury','testimony','bequest','bequeather','ordinances','deviation','animosity','forbearing','scribe','discord','bestower','penalty','heaped-up','swift','arbitrate','deluded','assemble','sovereignty','prudence','expelled','abstaining','disputed','leaper','hereafter','supplicate','invoke','equitable','inclining','faction','falsehood','oaths','alter','pious','abiding','repent','attain','prostrating','hasten','ally','intimates','hardship','ally','reinforce','tidings','thereby','decree','afflicted','refuge','disputing','distress','martyred','uninhibited','stationed','persevere','immorality','steadfast','martyrdom','forsake','attributable','booty','manifest']
     if word in list:
      print('Complex/Archaic Word:',word)




for i in range(10):
  
  print("Archaic Translation:",seq2text(X_test[i]))
  print("Expected Universal Translation:",seq2summary(y_test[i]))
  print("Predicted Universal Translation:",decode_sequence(X_test[i].reshape(1,291)).lstrip())
  x=decode_sequence(X_test[i].reshape(1,291)).lstrip()
  x1=x.split()
  for a in range(len(x1)): 
    checkdifficultwordprobability(x1[a]) 

  object1 = ("Devil","Angels","Prophets","Believers","Non-Believers")
  object2 = ("lost", "astonished", "happy","confused","hopleless") 
  object3 = ("guided", "fooled","mocked","cursed","helped")

  num = random.randrange(0,5)
  num1 = random.randrange(0,5)
  num2 = random.randrange(0,5)
  attack=("And"+" "+object1[num]+" "+"found you"+" "+object2[num1]+" "+"and"+" "+object3[num2]+" "+"you.")

  B1=str(decode_sequence(X_test[i].reshape(1,291)).lstrip())
  B2=str(seq2summary(y_test[i]))
  C1=B1
  C2=B2
  q=decode_sequence(X_test[i].reshape(1,291)).lstrip()
  s = SequenceMatcher(None, C1,C2)

  regex = re.compile(r'\b(' + '|'.join(map(re.escape, dct.keys())) + r')\b')
  # with attack
  print("Text Style Transfer Translation:",attack)
  a = set(C1.split())
  #b = set(regex.sub(lambda match: dct[match.group(0)], q).split())
  b=set(attack.split())
  #jj=str(regex.sub(lambda match: dct[match.group(0)], q))
  jj=str(attack)
  similarity1 = float(len(a.intersection(b))*1.0/len(a.union(b)))
  document=[B1,jj]


  predictions, raw_output  = model.predict([attack])
  # without attack
  #print("Text Style Transfer Translation:",regex.sub(lambda match: dct[match.group(0)], q))
  #predictions, raw_output  = model.predict([regex.sub(lambda match: dct[match.group(0)], q)])
  print('937 Adversarial Attack on Text Style Transfer Translation Yes[1]/No[0]:',predictions)
  if str(predictions) in ['[1]']:
   sum122=sum122+1

 
 
  print('BLEUscore:',bleu([C1], C2, smoothing_function=smoothie))
  print('SequenceMatcher:',s.ratio())
  print('Jaro Distance:',jellyfish.jaro_similarity(C1, C2))
  print('Match Rating Approach Comparison:',jellyfish.match_rating_comparison(C1, C2))
  print('Hamming Distance:',textdistance.hamming.normalized_similarity(C1,C2))
  print('Normalized Levenshtein:',normalized_levenshtein.similarity(C1, C2))
  print('Jaccard Similarity',similarity1)
  vectorizer = TfidfVectorizer()
  trsfm=vectorizer.fit_transform(document)
 
  print('Cosine Similarity:',cosine_similarity(trsfm[0], trsfm[1]))
  print(i)
  print("\n")

  sum=sum+bleu([C1], C2, smoothing_function=smoothie)
  sum1=sum1+s.ratio()
  sum2=sum2+jellyfish.jaro_similarity(C1, C2)
  sum3=sum3+jellyfish.match_rating_comparison(C1, C2)
  sum4=sum4+textdistance.hamming.normalized_similarity(C1,C2)
  sum5=sum5+similarity1
  sum6=sum6+normalized_levenshtein.similarity(C1, C2)
  sum7=sum7+cosine_similarity(trsfm[0], trsfm[1])


performance=sum/10
performance1=sum1/10
performance2=sum2/10
performance3=sum3/10
performance4=sum4/10
performance5=sum5/10
performance6=sum6/10
performance7=sum7/10
print('Syntactic Analysis')
print('BLEUscore Performance:',performance*100,'%')
print('SequenceMatcher Performance:',performance1*100,'%')
print('Jaro Similarity Performance:',performance2*100,'%')
print('Match Rating Comparison Performance:',performance3*100,'%')
print('Hamming Distance Performance:',performance4*100,'%')
print('Normalized Levenshtein Performance:',performance6*100,'%')
print('Semantic Analysis')
print('Jaccard Similarity Performance:',performance5*100,'%')
print('Cosine Similarity Performance:',performance7*100,'%')
average122=sum122/10
print('937 Adversarial Attack Predictions Average:',average122*100,'%')


   

Archaic Translation: though he puts forth his excuses 
Expected Universal Translation: even if he presents his excuses 
Predicted Universal Translation: while he left it as if he was to each straight
Text Style Transfer Translation: And Devil found you confused and fooled you.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

937 Adversarial Attack on Text Style Transfer Translation Yes[1]/No[0]: [1]
BLEUscore: 0.17668503765114876
SequenceMatcher: 0.38461538461538464
Jaro Distance: 0.5772731193926847
Match Rating Approach Comparison: False
Hamming Distance: 0.08695652173913049
Normalized Levenshtein: 0.26086956521739135
Jaccard Similarity 0.0
Cosine Similarity: [[0.]]
0


Archaic Translation: to moses and aaron we granted the criteria of discerning right from wrong and we gave them the light and a reminder to the pious ones 
Expected Universal Translation: and we had already given moses and aaron the criterion and a light and a reminder for the righteous 
Predicted Universal Translation: and moses said to his people and we were of the children of israel and the leper with my signs and you were of the polytheists
Complex/Archaic Word: polytheists
Text Style Transfer Translation: And Non-Believers found you hopleless and fooled you.


  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

937 Adversarial Attack on Text Style Transfer Translation Yes[1]/No[0]: [1]
BLEUscore: 0.1061608096958646
SequenceMatcher: 0.256
Jaro Distance: 0.6524811581440496
Match Rating Approach Comparison: False
Hamming Distance: 0.011695906432748537
Normalized Levenshtein: 0.26900584795321636
Jaccard Similarity 0.03225806451612903
Cosine Similarity: [[0.17721061]]
8


Archaic Translation: is it you who cause it from the rainclouds to come down or are we the of it to come down 
Expected Universal Translation: is it you who brought it down from the clouds or is it we who bring it down 
Predicted Universal Translation: is it you who brought it down from the clouds or is it we who bring it down
Text Style Transfer Translation: And Believers found you confused and guided you.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

937 Adversarial Attack on Text Style Transfer Translation Yes[1]/No[0]: [1]
BLEUscore: 0.9865741364867494
SequenceMatcher: 0.9933774834437086
Jaro Distance: 0.9956140350877193
Match Rating Approach Comparison: True
Hamming Distance: 0.9868421052631579
Normalized Levenshtein: 0.9868421052631579
Jaccard Similarity 0.05263157894736842
Cosine Similarity: [[0.05366558]]
9


Syntactic analysis
BLEUscore Performance: 64.61219201673674 %
SequenceMatcher Performance: 69.56543333152658 %
Jaro Similarity Performance: 83.69037402237535 %
Match Rating Comparison Performance: 70.0 %
Hamming Distance Performance: 53.475705680749726 %
Normalized Levenshtein Performance: 66.83785061386807 %
Semantic Analysis
Jaccard Similarity Performance: 4.64693427297619 %
Cosine Similarity Performance: [[12.19982158]] %
937 Adversarial Attack Predictions Average: 100.0 %


Encoder-Decoder Vigilance Model

In [1]:
import os
import re
import pandas as pd

In [2]:
train = pd.read_excel("937 Excel.xlsx")

#Reducing the training sample for fast execution
train = train.sample(frac = 0.2)

#splitting the training set in to training and validation sets
from sklearn.model_selection import train_test_split
train, val =  train_test_split(train, test_size = 0.2, random_state = 120)

In [3]:
len(val)

13

In [4]:
train.head()

,Translation,Label
104,And Non-Believers found you lost and helped you.,1
283,those who spend their wealth in allah s way by...,0
192,and mention when we made the house a place of ...,0
182,for those who swear not to have sexual relatio...,0
196,and for their women is a provision according t...,0


In [5]:
val.head()

,Translation,Label
289,except for those who feel remorse correct them...,0
237,so how will it be when we gather them for a da...,0
70,And Prophets found you hopeless and guided you.,1
226,indeed those whom the angels take in death whi...,0
247,except for those who feel remorse after that a...,0


In [6]:
train.shape

(52, 2)

In [7]:
val.shape

(13, 2)

In [12]:
!pip install tokenizers==0.10.1

     |████████████████████████████████| 3.2MB 4.3MB/s 
  Found existing installation: tokenizers 0.9.4
    Uninstalling tokenizers-0.9.4:
      Successfully uninstalled tokenizers-0.9.4


In [13]:
!pip install simpletransformers

In [14]:
from simpletransformers.classification import ClassificationModel

#Create a ClassificationModel
model = ClassificationModel('roberta', 'roberta-base', num_labels=2, use_cuda = False)

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [15]:
model.train_model(train)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:610: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/52 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/7 [00:00<?, ?it/s]

(7, 0.6740647894995553)

In [18]:
#Evaluating With F1 Score & Accuracy

from sklearn.metrics import f1_score, accuracy_score
def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')

In [23]:
import pandas as pd
from sklearn.model_selection import train_test_split
import string
from string import digits
import re
from sklearn.utils import shuffle
from keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM, Input, Dense,Embedding
from keras.models import Model,load_model
#from keras.utils import plot_model
from keras.preprocessing.text import one_hot
from keras.preprocessing.text import Tokenizer
from keras.models import model_from_json
import pickle as pkl
import numpy as np

In [24]:
# loading the model architecture and asigning the weights
json_file = open('model_2.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model_loaded = model_from_json(loaded_model_json)
# load weights into new model
model_loaded.load_weights("model_weight_5.h5")

In [25]:
latent_dim = 500
#inference encoder
encoder_inputs_inf = model_loaded.input[0] #Trained encoder input layer
encoder_outputs_inf, inf_state_h, inf_state_c = model_loaded.layers[4].output # retoring the encoder lstm output and states
encoder_inf_states = [inf_state_h,inf_state_c]
encoder_model = Model(encoder_inputs_inf,encoder_inf_states)

In [27]:
#inference decoder
# The following tensor will store the state of the previous timestep in the "starting the encoder final time step"
decoder_state_h_input = Input(shape=(latent_dim,)) #becase during training we have set the lstm unit to be of 50
decoder_state_c_input = Input(shape=(latent_dim,))
decoder_state_input = [decoder_state_h_input,decoder_state_c_input]

# # inference decoder input
decoder_input_inf = model_loaded.input[1] #Trained decoder input layer
# decoder_input_inf._name='decoder_input'
decoder_emb_inf = model_loaded.layers[3](decoder_input_inf)
decoder_lstm_inf = model_loaded.layers[5]
decoder_output_inf, decoder_state_h_inf, decoder_state_c_inf = decoder_lstm_inf(decoder_emb_inf, initial_state =decoder_state_input)
decoder_state_inf = [decoder_state_h_inf,decoder_state_c_inf]
#inference dense layer
dense_inf = model_loaded.layers[6]
decoder_output_final = dense_inf(decoder_output_inf)# A dense softmax layer to generate prob dist. over the target vocabulary

decoder_model = Model([decoder_input_inf]+decoder_state_input,[decoder_output_final]+decoder_state_inf)

In [28]:
with open('NMT_data.pkl','rb') as f:
  X_train, y_train, X_test, y_test = pkl.load(f)

In [29]:
with open('tokenizer_input.pkl','rb') as f:
  tokenizer_input = pkl.load(f)
with open('tokenizer_target.pkl','rb') as f:
  tokenizer_target = pkl.load(f)
# Creating the reverse mapping to get the word from the index in the sequence
reverse_word_map_input = dict(map(reversed, tokenizer_input.word_index.items()))
reverse_word_map_target = dict(map(reversed, tokenizer_target.word_index.items()))

In [30]:
# Code to predct the input sentences translation
def decode_seq(input_seq):
  # print("input_seq=>",input_seq)
  state_values_encoder = encoder_model.predict(input_seq)
  # intialize the target seq with start tag
  target_seq = np.zeros((1,1))
  target_seq[0, 0] = tokenizer_target.word_index['start']
  # print("target_seq:=>",target_seq)
  stop_condition = False
  decoder_sentance = ''
  # print("Beforee the while loop")
  while not stop_condition:
    sample_word , decoder_h,decoder_c= decoder_model.predict([target_seq] + state_values_encoder)
    # print("sample_word: =>",sample_word)
    sample_word_index = np.argmax(sample_word[0,-1,:])
    # print("sample_word_index: ",sample_word_index)
    decoder_word = reverse_word_map_target[sample_word_index]
    decoder_sentance += ' '+ decoder_word
    # print("decoded word:=>",decoder_word)
    # print(len(decoder_sentance))
    # print("len(decoder_sentance) > 70: ",len(decoder_sentance) > 70)
    # print('decoder_word == "end"',decoder_word == 'end')
    # print(decoder_word == 'end' or len(decoder_sentance) > 70)
    # stop condition for the while loop
    if (decoder_word == 'end' or 
        len(decoder_sentance) > 2000):
        stop_condition = True
        # print("from if condition")
    # target_seq = np.zeros((1,1))
    target_seq[0, 0] = sample_word_index
    # print(target_seq)
    state_values_encoder = [decoder_h,decoder_c]
  return decoder_sentance

In [31]:
!pip install jellyfish

     |████████████████████████████████| 92kB 3.7MB/s 


In [32]:
!pip install textdistance

In [33]:
!pip install strsimpy

     |████████████████████████████████| 51kB 2.8MB/s 


In [35]:
import re
import random
import jellyfish
import scipy
import textdistance
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from nltk.translate import bleu
from strsimpy.normalized_levenshtein import NormalizedLevenshtein
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from difflib import SequenceMatcher
from nltk.translate.bleu_score import SmoothingFunction
count_vect = CountVectorizer()
cc = SmoothingFunction() 
smoothie = SmoothingFunction().method4
sum=0
sum1=0
sum2=0
sum3=0
sum4=0
sum5=0
sum6=0
sum7=0
sum122=0
average122=0
performance=0
performance1=0
performance2=0
performance3=0
performance4=0
performance5=0
performance6=0
performance7=0
normalized_levenshtein = NormalizedLevenshtein()
dct = {'deviant':'devaint (departing from usual or accepted standards)',
       'cessation':'end',
    'conspicuous':'visible',
    'repelling':'repelling (turning it back)',
    'calamity':'disaster',
    'merchandise':'goods',
    'impassioned':'imapassioned (filled with or showing great emotion)',
    'coercion':'force',
    'despaired':'lost hope',
    'adversity':'disaster',
    'exultant':'thrilled',
    'adorments':'glitter',
    'unto':'to',
    'depositories':'depositories (a place where things can be stored)',
    'anchorage':'anchorage (a place where the boat is or can be anchored)',
    'disembark':'disembark (to leave a ship after a journey)',
    'forelock':'forelock (a piece of hair that grows or falls over the forehead)',
    'obstinate':'aggressive',
    'tyrant':'cruel ruler',
    'disquieting':'alarming',
    'hamstrung':'killed',
    'shriek':'shriek (a piercing cry or blast from the sky)',
    'apprehension':'harm',
    'availed':'profited',
    'bestowal':'gift',
    'undiminished':'fully',
    'impending':'impending (near)',
    'tyranny':'oppression',
    'enmity':'extreme hatred',
    'ambush':'attack',
    'besiege':'surround',
    'kinship':'relationship',
    'striven':'striven (make great efforts to achieve something)',
    'commerce':'trade',
    'flanks':'flanks (the side of a persons or animals body between the ribs and the hips)',
    'tranquility':'calm',
    'apprehensive':'anxious',
    'vanities':'idle talks',
    'stingy':'not generous',
    'bedouins':'bedouins (a member of an Arab people living in or near the desert)',
    'invocations':'prayers',
    'afflicted':'suffered',
    'inflictions':'imposed',
    'wherein':'in which',
    'therewith':'therewith (soon or immediately after that)',
    'hostilities':'hostilities (hatred)',
    'hostility':'hostility (hatred)',
    'obstruct':'block',
    'pledge':'pledge(word of honor)',
    'inflicts':'imposes',
    'massacre':'massacre (murder/slaughter of people) ',
    'massacred':'massacred (murder/slaughter of people)',
    'commodities':'assets',
    'debased':'wicked',
    'eminent':'eminent (very important)',
    'defilement':'degradation',
    'insolent':'arrogant',
    'insolently':'arrogantly',
    'prone':'prone (lying flat)',
    'afflict':'suffer',
    'serpent':'snake',
    'bewitched':'bewitched (to affect by witchcraft)',
    'subjugators':'winners',
    'tablets':'tablets (a flat slab, intended to bear writtings)',
    'heedless':'careless',
    'inscription':'inscription (a historical, religious, or other record cut, impressed, painted, or written on stone, brick, metal, or other hard surface)',
    'absolved':'exempt',
    'contrived':'manipulated',
    'equate':'regard',
    'thereform':'from that place',
    'deities':'Gods',
    'intercessor':'mediator',
    'intercessors':'mediators',
    'finality':'finality (the fact of being final)',
    'appraisal':'assessment',
    'prohibited':'forbid',
    'digress':'digress (to turn aside)',
    'tyrannical':'brutal',
    'thereto':'to that',
    'expiation':'expiation (to make amends)',
    'perjury':'perjury (lying under oath)',
    'sanctity':'holiness',
    'substitute':'alternative',
    'devour':'swallow',
    'detriment':'disadvantage',
    'ordains':'establish',
    'ordained':'established',
    'decree':'legal order',
    'decreed':'mandated',
    'chastity':'purity',
    'affliction':'suffering',
    'arbitrator':'arbitrator (settler of a dispute)',
    'reconciliation':'harmony',
    'boastful':'bragging',
    'stinginess':'extreme greed',
    'fabricated':'constructed',
    'far astray':'far away',
    'accommodation':'compromise',
    'admonish':'advise',
    'attainment':'achievement',
    'lofty':'high',
    'disposer':'giver',
    'intercedes':'interfere',
    'emigrate':'emigrate (abandon their homes)',
    'wherein':'in which',
    'acquisitions':'profits',
    'conferred':'awarded',
    'incumbent':'necessary',
    'prostrated':'bowed',
    'slander':'falsehood',
    'recompensed':'repayed',
    'contempt':'disrespect',
    'evasion':'avoidance',
    'his abundance':'his rewards',
    'wavering':'swaying',
    'pardoning':'forgiving',
    'pardoned':'forgiven',
    'discriminate':'differentiate',
    'solemn':'dignified',
    'avert':'turn away',
    'desist':'desist (stop doing something)',
    'disdained':'disrespected',
    'emigrant':'emigrant (one who leaves his own country)',
    'manifest': 'clear',
    'attributable': '[attributable (considered impossible)]',
    'martyrdom': 'martyrdom (killed in the cause of God)',
    'steadfast': 'firm',
    'an immorality': 'a crime',
    'persevere': 'work hard',
    'stationed': 'firm',
    'uninhibited': 'unrestrained',
    'martyred': 'martyred (killed in the cause of God)',
    'distress': 'pain',
    'disputing': 'arguing',
    'refuge': 'place to hide',
    'afflicted': 'depressed',
    'decree determined': 'term appointed',
    'thereby': 'by that',
    'tidings': 'news',
    'reinforce': 'increase',
    'hardships': 'severe suffering',
    'intimates': 'close friends',
    'hasten': 'hurry',
    'prostrating': 'bowing',
    'attain': 'achieve',
    'repent': 'feel remorse',
    'abiding': 'living',
    'their recompense': 'their reward',
    'pious': 'devoted',
    'alter': 'change',
    'oaths': 'promises',
    'falsehood': 'untruthfulness',
    'faction': 'group',
    'inclining': 'favoring',
    'equitable': 'shared',
    'invoke': 'call on',
    'supplicate': 'beg',
    'hereafter': 'life after death',
    'leaper': 'leper (A person affected with leprosy disease)',
    'disputed': 'argued',
    'abstaining': 'abstaining (to hold oneself back voluntarily)',
    'expelled': 'rejected',
    'prudence': 'fear',
    'sovereignty': 'power',
    'assemble': 'gather',
    'deluded': 'fooled',
    'arbitrate': 'achieve settlement',
    'swift': 'very fast',
    'heaped-up': 'stacked',
    'penalty': 'punishment',
    'bestower': 'granter',
    'scribe': 'scribe (the one who makes a written copy professionally)',
    'forbearing': 'tolerant',
    'animosity': 'extreme hatred',
    'deviation' :'change',
    'ridicule': 'make fun of',
    'ordinances': 'law',
    'bequeather': 'will',
    'bequest': 'will',
    'testimony': 'testimony (declaration about truth)',
    'usury': 'lending money at high interest',
    'admonition': 'caution',
    'kursi': 'kursi (Chair)',
    'deity': 'god',
    'kingship': 'nobility',
    'acquainted': 'aware',
    'transgress': 'does wrong',
    'alleviation': 'relief',
    'retribution': 'compensation',
    'prescirbed': 'recommended',
    'dissension': 'disagreement',
    'reprieved': 'relief of blame',
    'exalted': 'praised',
    'all encompassing': 'All-Sufficient',
    'dominion': 'authority',
    'abrogate': 'put an end',
    'bounty': 'gift',
    'enjoins': 'commands',
    'wrath': 'extreme anger',
    'wretched': 'terrible',
    'resurrection': 'resurrection (Awakening from the dead)',
    'resurrected':'resurrected (Awakened from the dead)',
    'aggression': 'attack',
    'evicting': 'throwing out',
    'evict': 'throw out',
    'enjoining': 'commanding',
    'woe': 'suffering',
    'unlettered': 'vulgar',
    'covet': 'wish',
    'concealing': 'hiding',
    'irrigate': 'water',
    'plow': 'dig',
    'median between that': '(it is) between the two conditions',
    'do you take us in ridicule?': 'are you mocking us?',
    'despised': 'I look down upon you',
    'transgressed': 'spread sin',
    'transgressing': 'wrong-doers',
    'gushed':'poured out',
    'aided': 'helped',
    'intercession': 'appeal',
    'suffice': 'be enough',
    'righteousness': 'goodness',
    'discord': 'disharmony',
    'ally': 'protector',
    'deviation': 'change',
     '<end>': ' ',
     'discord': 'disharmony',
     'forsake': 'abandon',
     'sovereign': 'ruler',
     'recompense': 'Judgement',
     'bestowed': 'granted',
     'evoked': 'awaken',
     'astray':'lost',
    'transgressions':'wrong-doings',
    'kindled': 'start',
    'illuminated': 'lit up',
    'dread': 'fear',
    'is encompassing of': 'has surrounded',
    'competent': 'capable',
    'attribute': 'assign',
    'abide': 'live',
    'eternally': 'forever',
    'defiantly': '',
    'covenant': 'agreement',
    'sever': 'to cut off',
    'santify': 'purify',
    'prostrate': 'bow',
    'therein': 'in there',
    'conceal': 'hide',
    'scorching':'very hot',
    'discern':'observe',
    'foliage':'foliage (plant leaves collectively)',
    'enumerate':'list',
    'excretion':'excretion (a process in which metabolic waste is eliminated from an organism)',
    'palatable':'palatable (pleasant to taste)',
    'appease':'make concessions',
    'envelopment':'utmost degree',
    'probed':'examined',
    'censured':'censured (severe criticism)',
    'insolvent':'broke',
    'exultantly':'happily',
    'banished':'expelled',
    'sublimity':'dignity',
    'aversion':'dislike',
    'meridian':'meridian (a great circle of the earth passing through the poles and any given point on the earths surface)',
    'deviance':'deviance (abnormality)',
    'murky':'unclear',
    'scalds':'burns',
    'fatigue':'tiredness',
    'supplement':'option',
    'supplication':'supplication (an act of asking earnestlt or humbly)',
    'adhere':'hold fast',
    'unchaste':'shameless',
    'abstention':'restrain',
    'invocation':'calling',
    'insolence':'arrogance',
    'inevitably':'unavoidably',
    'atrocious':'horrifyingly wicked',
    'slacken':'slacken (become less active)',
    'manna':'manna (the food miraculously supllied to the Israelities in the wilderness)',
    'hastened':'hurried',
    'o':'', #'o (o - used before a name in direct address)',
    'splendor':'beauty',
    'heedlessness':'careless',
    'thereupon':'therefore',
    'convulsion':'earthquake',
    'synagogues':'synagogues (a place of assembly (temple))',
    'monasteries':'monasteries (a building occupied by monks living under religious vows)',
    'tyrannized':'oppressed',
    'servitude':'servitude (the state of being a slave)',
    'outstrip':'overtake',
    'incitements':'incitements (provoking unlawful behaviour)',
    'fornicator':'fornicator (a person who has sex with someone who they are not married to)',
    'chaste':'chaste (to stay away from sexual intercourse)',
    'emancipation':'freedom',
    'compulsion':'compulsion (a very strong desire to do something)',
    'adhering':'adhering (to hold fast)',
    'sparingly':'cautiously',
    'tyrants':'cruel rulers',
    'kindred':'family and tribe',
    'haughtiness':'scronful arrogance',
    'haughty':'scornfully arrogant',
    'amending':'amending(to alter)',
    'amenders':'amenders (those who can alter)',
    'supremacy':'high rank',
    'sanctuary':'sanctuary (a holy place',
    'amply':'plenty',
    'replenished':'refill',
    'trodden':'trodden (walk in a specific way)',
    'adornment':'luxury',
    'enormity':'enormity (extreme scale)',
    'dignitaries':'dignitaries (high rank people)',
    'pliable':'flexible',
    'tamarisk':'tamarisk (flowering plant)',
    'benefactor':'benefactor (a person who gives some form of help to benefit a person)',
    'deluded':'fooled',
    'perish':'die',
    'extinguished':'furnished',
    'adorned':'furnished',
    'obliterated':'destroyed',
    'pelted':'struck',
    'contemptible':'humiliated',
    'scalding':'burning',
    'exertion':'labour',
    'explicit':'clear',
    'begotten':'begotten (brought into existence a child)',
    'lineage':'lineage (direct descent from an ancestor)',
    'depletion':'reduction',
    'purulence':'purulence (dirty wound discharges)',
    'reiterate':'reiterate (to do over repeatedly)',
    'skeptic':'skeptic (an attitude of doubt)',
    'skepticism':'skepticism (an attitude of doubt)',
    'exult':'rejoice',
    'decisive word':'word of judgement',
    'thereof':'of that',
    'enjoined':'urged',
    'thereby':'by that means',
    'subdued':'accomplished',
    'apportioned':'divided',
    'conjunction':'conjunction (state of being conjoined)',
    'denominations':'groups',
    'averse':'averse (feeling of dislike)',
    'transcribed':'transcribed (to make a written copy of)',
    'uff':'uff (an expression of distaste and irritation)',
    'oblivion':'oblivion (lack of consciousness)',
    'vengeance':'revenge',
    'allegiance':'loyalty',
    'imminent':'imminent (ready to take place)',
    'chauvinism':'chauvinism (extreme ignorance)',
    'stalks':'stalks (a part of a plant that supports another)',
    'defiance':'resistance',
    'inscribed':'inscribed (object marked with characters)',
    'parchment':'parchment (material made from animal skin)',
    'soothsayer':'fortune teller',
    'deterrence':'deterrrence (to prevent someone from doing something)',
    'sheaths':'enclosing',
    'brocade':'brocade (a rich fabric woven with a raised pattern, typically with gold or silver thread)',
    'pavilions':'in very large tents',
    'affluence':'wealth',
    'fortified':'armed',
    'propped':'leaned',
    'lodge them':'lodge them (let the women live (in iddat)',
    'dwell':'live',
    'retain':'keep',
    'dissolution':'formal release from guilt',
    'alienation':'isolation',
    'aloft':'up',
    'snout':'nose',
    'infirm':'weak',
    'cubits':'cubits (ancient measurement of length)',
    'aorta':'aorta (the main artery of the blood, supplying oxygenated blood to the circulatory system)',
    'supplicant':'supplicant (someone who prays to God)',
    'vainly':'unsuccessfully',
    'ascent':'rising upward',
    'grandeur':'glory',
    'arduous':'severe',
    'respite':'delay',
    'convulse':'shake violently',
    'scowled':'scowled (to contract the brow in an expression of displeasure)',
    'permanence':'permanence (the quality of being permanent)',
    'contorted':'deformed',
    'swaggering':'swaggering (overbearing self-connfidence)',
    'mirage':'mirage (an image, produced by very hot air, of something that seems to be far away but does not really exist)',
    'entwined':'thick',
    'pasture':'grass',
    'torrents':'sudden large amount',
    'herbage':'nutricious plants',
    'shrubbery':'group of shrubs',
    'derisive':'mocking',
    'jesting':'joking',
    'musk':'fragrance',
    'trench':'ditch',
    'slate':'rock',
    'stubble':'rotten rubbish',
    'discernment':'judgement',
    'polytheists':'polytheists (belief in many Gods)',
    'abyss':'bottomless pit'

}

def dictionary1(word):
     list=['abyss','polytheists','resurrected','discernment','unto','stubble','trench','slate','musk','o','derisive','jesting','torrents','herbage','shrubbery','pasture','mirage','entwined','permanence','contorted','swaggering','scowled','respite','convulse','arduous','grandeur','supplicant','ascent','vainly','infirm','cubits','aorta','snout','alienation','aloft','dissolution','lodge','retain','dwell','propped','fortified','affluence','sheaths','brocade','pavilions','deterrence','defiance','allegiance','imminent','chauvinism','offshoots','stalks','vengeance','oblivion','Uff','transcribed','thereby','subdued','apportioned','conjunction','denominations','averse','enjoined','thereof','decisive word','skeptic','skepticism','shackle','shackles','reiteration','depletion','purulence','pelted','contemptible','scalding','exertion','explicit','begotten','lineage','extinguished','adorned','obliterated','deluded','perish','tamarisk','benefactor','pliable','insolently','trodden','adornment','enormity','dignitaries','massacred','amply','replenished','countenance','intercessors','haughtiness','haughty','sanctuary','amending','amenders','supremacy','tyrant','kindred','adhering','sparingly','compulsion','emancipation','chaste','fornicator','servitude','outstrip','incitements','convulsion','synagogues','monasteries','tyrannized','heedlessness','thereupon','slacken','manna','hastened','splendor','supplication','adhere','unchaste','abstention','invocation','insolence','inevitably','atrocious','probed','censured','insolvent','exultantly','banished','sublimity','aversion','meridian','deviance','murky','scalds','fatigue','supplement','foliage','enumerate','excretion','palatable','envelopment','appease','scorching','discern','deviant','cessation','calamity','repelling','conspicuous','merchandise','impassioned','coercion','despaired','adversity','impending','undiminished','bestowal','availed','apprehension','shriek','hamstrung','disquieting','tyrant','obstinate','forelock','disembark','anchorage','depositories','adorments','exultant','tyranny','enmity','ambush','besiege','kinship','striven','commerce','flanks','tranquility','apprehensive','vanities','stingy','bedouins','invocations','afflicted','inflictions','wherein','therewith','hostilities','hostility','obstruct','pledge','inflicts','massacre','commodities','debased','eminent','defilement','insolent','prone','afflict','serpent','bewitched','subjugators','tablets','heedless','inscription','absolved','contrived','equate','thereform','deities','intercessor','finality','appraisal','prohibited', 'digress','tyrannical','thereto','expiation','perjury','sanctity','substitute','devour','detriment','ordains','ordained','decree','decreed','chastity','affliction','arbitrator','reconciliation','boastful','stinginess','fabricated','far astray','accommodation','admonish','attainment','lofty','disposer','intercedes', 'emigrate', 'wherein', 'acquisitions', 'conferred','incumbent', 'prostrated','slander','recompensed','contempt','evasion','his abundance','wavering','pardoning','pardoned','discriminate','solemn','avert','desist','disdained','emigrant','sovereign','recompense','bestowed','evoked','astray','transgressions','kindled','illuminated','dread','encompassing','competent','attribute','abide','eternally','defiantly','covenant','sever','sanctify','prostrate','therein','conceal','righteousness','suffice','intercession','aided','gushed','transgressions','transgressed','despised','ridicule','median','plow','irrigate','concealing','covet','unlettered','woe','enjoining','evict','evicting','aggression','resurrection','wretched','wrath','enjoins','bounty','abrogate','dominion','all encompassing','exalted','reprieved','dissension','prescribed','retribution','alleviation','transgresses','acquainted','kingship','deity','kursi','admonition','usury','testimony','bequest','bequeather','ordinances','deviation','animosity','forbearing','scribe','discord','bestower','penalty','heaped-up','swift','arbitrate','deluded','assemble','sovereignty','prudence','expelled','abstaining','disputed','leaper','hereafter','supplicate','invoke','equitable','inclining','faction','falsehood','oaths','alter','pious','abiding','repent','attain','prostrating','hasten','ally','intimates','hardship','ally','reinforce','tidings','thereby','decree','afflicted','refuge','disputing','distress','martyred','uninhibited','stationed','persevere','immorality','steadfast','martyrdom','forsake','attributable','booty','manifest']
     if word in list:
       print('Complex/Archaic Word:',word)

for i in range(10):
  if i==554:
    i=i+1
  sentence = X_test[i]
  original_target = y_test[i]
  input_seq = tokenizer_input.texts_to_sequences([sentence])
  pad_sequence = pad_sequences(input_seq, maxlen= 200, padding='post')
  # print('input_sequence =>',input_seq)
  # print("pad_seq=>",pad_sequence)
  predicted_target = decode_seq(pad_sequence)
  print("Archaic Translation:",sentence)
  print("Expected Universal Translation:",original_target[6:-4])
  print("Predicted Universal Translation:",predicted_target[:-4])
  x=predicted_target[:-4]
  x1=x.split()
  for a in range(len(x1)): 
    dictionary1(x1[a]) 

  object1 = ("Devil","Angels","Prophets","Believers","Non-Believers")
  object2 = ("lost", "astonished", "happy","confused","hopleless") 
  object3 = ("guided", "fooled","mocked","cursed","helped")

  num = random.randrange(0,5)
  num1 = random.randrange(0,5)
  num2 = random.randrange(0,5)
  attack=("And"+" "+object1[num]+" "+"found you"+" "+object2[num1]+" "+"and"+" "+object3[num2]+" "+"you.")


  B1=str(predicted_target[:-4])
  B2=str(original_target[6:-4])
  C1=B1
  C2=B2
  q=predicted_target[:-4]
  s = SequenceMatcher(None, C1,C2)
  regex = re.compile(r'\b(' + '|'.join(map(re.escape, dct.keys())) + r')\b')
 
  # with attack
  #print("Text Style Transfer Translation:",attack)
  #a = set(C1.split())
  #b = set(regex.sub(lambda match: dct[match.group(0)], q).split())
  #b=set(attack.split())
  #jj=str(regex.sub(lambda match: dct[match.group(0)], q))
  #jj=str(attack)
  #similarity1 = float(len(a.intersection(b))*1.0/len(a.union(b)))
  #document=[B1,jj]


  #predictions, raw_output  = model.predict([attack])
  # without attack
  print("Text Style Transfer Translation:",regex.sub(lambda match: dct[match.group(0)], q))
  predictions, raw_output  = model.predict([regex.sub(lambda match: dct[match.group(0)], q)])
  print('937 Adversarial Attack on Text Style Transfer Translation Yes[1]/No[0]:',predictions)
  if str(predictions) in ['[1]']:
    sum122=sum122+1



  '''print("Text Style Transfer Translation:",regex.sub(lambda match: dct[match.group(0)], q))
  a = set(C1.split())
  b = set(regex.sub(lambda match: dct[match.group(0)], q).split())
  jj=str(regex.sub(lambda match: dct[match.group(0)], q))
  similarity1 = float(len(a.intersection(b))*1.0/len(a.union(b)))
  document=[B1,jj]'''
  print('BLEUscore:',bleu([C1], C2, smoothing_function=smoothie))
  print('SequenceMatcher:',s.ratio())
  print('Jaro Distance:',jellyfish.jaro_similarity(C1, C2))
  #print('Match Rating Approach Comparison:',jellyfish.match_rating_comparison(C1, C2))
  print('Hamming Distance:',textdistance.hamming.normalized_similarity(C1,C2))
  print('Normalized Levenshtein:',normalized_levenshtein.similarity(C1, C2))
  print('Jaccard Similarity',similarity1)
  vectorizer = TfidfVectorizer()
  trsfm=vectorizer.fit_transform(document)
 
  print('Cosine Similarity:',cosine_similarity(trsfm[0], trsfm[1]))
  pd.DataFrame(trsfm.toarray(),columns=vectorizer.get_feature_names(),index=['Predicted Universal Translation','Text Style Transfer Translation'])
  print(i)
  print("\n")
  
  sum=sum+bleu([C1], C2, smoothing_function=smoothie)
  sum1=sum1+s.ratio()
  sum2=sum2+jellyfish.jaro_similarity(C1, C2)
  #sum3=sum3+jellyfish.match_rating_comparison(C1, C2)
  sum4=sum4+textdistance.hamming.normalized_similarity(C1,C2)
  sum5=sum5+similarity1
  sum6=sum6+normalized_levenshtein.similarity(C1, C2)
  sum7=sum7+cosine_similarity(trsfm[0], trsfm[1])

performance=sum/10
performance1=sum1/10
performance2=sum2/10
#performance3=sum3/3742
performance4=sum4/10
performance5=sum5/10
performance6=sum6/10
performance7=sum7/10
print('Syntactic analysis')
print('BLEUscore Performance:',performance*100,'%')
print('SequenceMatcher Performance:',performance1*100,'%')
print('Jaro Similarity Performance:',performance2*100,'%')
#print('Match Rating Comparison Performance:',performance3*100,'%')
print('Hamming Distance Performance:',performance4*100,'%')
print('Normalized Levenshtein Performance:',performance6*100,'%')
print('Semantic Analysis')
print('Jaccard Similarity Performance:',performance5*100,'%')
print('Cosine Similarity Performance:',performance7*100,'%')

average122=sum122/10
print('937 Adversarial Attack Predictions Average:',average122*100,'%')   

Archaic Translation: have you seen the one who has chosen his desires as his lord god has knowingly caused him to go astray sealed his ears and heart and veiled his vision who besides god can guide him will they then not take heed
Expected Universal Translation: have you seen he who has taken as his god his own desire and allah has sent him astray due to knowledge and has set a seal upon his hearing and his heart and put over his vision a veil so who will guide him after allah  then will you not be reminded
Predicted Universal Translation:  have you seen who is the protector and he has taken from his heart o muhammad considered by one who is able to speak merciful but when he has encompassed his brother has not been among them indeed he is powerful and a liar
Complex/Archaic Word: o
Text Style Transfer Translation:  have you seen who is the protector and he has taken from his heart  muhammad considered by one who is able to speak merciful but when he has encompassed his brother has not

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

937 Adversarial Attack on Text Style Transfer Translation Yes[1]/No[0]: [0]
BLEUscore: 0.4452037197623742
SequenceMatcher: 0.07203389830508475
Jaro Distance: 0.739475544975572
Hamming Distance: 0.09236947791164662
Normalized Levenshtein: 0.321285140562249
Jaccard Similarity 0.05555555555555555
Cosine Similarity: [[0.16499519]]
0


Archaic Translation: we mete out death among you and we are not to be outrun
Expected Universal Translation: we have decreed death among you and we are not to be outdone
Predicted Universal Translation:  we will inherit him in paradise and we have not granted them except an punished
Text Style Transfer Translation:  we will inherit him in paradise and we have not granted them except an punished


  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

937 Adversarial Attack on Text Style Transfer Translation Yes[1]/No[0]: [0]
BLEUscore: 0.30835634548031315
SequenceMatcher: 0.5476190476190477
Jaro Distance: 0.6653097062579821
Hamming Distance: 0.04166666666666663
Normalized Levenshtein: 0.41666666666666663
Jaccard Similarity 0.05555555555555555
Cosine Similarity: [[0.16499519]]
8


Archaic Translation: your lord certainly knows whatever their hearts conceal or reveal
Expected Universal Translation: and indeed your lord knows what their breasts conceal and what they declare
Predicted Universal Translation:  and indeed your lord knows what their breasts conceal and what they declare
Complex/Archaic Word: conceal
Text Style Transfer Translation:  and indeed your lord knows what their breasts hide and what they declare


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

937 Adversarial Attack on Text Style Transfer Translation Yes[1]/No[0]: [0]
BLEUscore: 0.9867551618071956
SequenceMatcher: 0.9933774834437086
Jaro Distance: 0.8489473684210527
Hamming Distance: 0.013157894736842146
Normalized Levenshtein: 0.9868421052631579
Jaccard Similarity 0.05555555555555555
Cosine Similarity: [[0.16499519]]
9


Syntactic analysis
BLEUscore Performance: 36.47504562201244 %
SequenceMatcher Performance: 39.67983868998633 %
Jaro Similarity Performance: 70.92812172041491 %
Hamming Distance Performance: 5.343131165015224 %
Normalized Levenshtein Performance: 38.8778510030365 %
Semantic Analysis
Jaccard Similarity Performance: 5.555555555555556 %
Cosine Similarity Performance: [[16.49951942]] %
937 Adversarial Attack Predictions Average: 0.0 %
